In [1]:
%run __init__.ipynb

Successfully connected to MongoDB


In [2]:
%%time

continue_script = True
# Init script
def init_script():
    file_path = os.path.join(OUTPUT_FOLDER, '107_FRANCE.csv')
    if path.exists(file_path):
        df_check = pd.read_csv(file_path, sep=';', low_memory=False)
        date_check = df_check[df_check['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y')]
        if len(date_check) > 0:
            print(f'Data already updated !')
            continue_script = False

init_script()

CPU times: user 192 µs, sys: 26 µs, total: 218 µs
Wall time: 5.67 ms


In [3]:
%%time

# Load Data Source
def load_source():
    file_path = os.path.join(OUTPUT_FOLDER,'107.csv')
    df = pd.DataFrame()
    
    if path.exists(file_path):
        df = pd.read_csv(file_path,sep=";")
        df = df.drop('Unnamed: 0',axis=1)
        df = df[(df['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y'))].reset_index(drop=True)
        if len(df) == 0:
            print(f'Data not updated !')
            continue_script = False
        else:
            df_save(df,'107_FRANCE','csv')
    return df


# Trasnform Data Source
def transform_data(df):
    if not df.empty:
        df = pd.merge(df, REF_FRANCE, left_on='CHILD', right_on='DEP_FULL', how='left'
                     ).rename(index=str, columns={"DEP_NAME":"LABEL"}
                             ).drop(['DEP_CODE','DEP_FULL','REG_NAME'], axis=1)

        df.loc[df['PARENT'] == 'root', 'LABEL'] = df['CHILD']
        df = df.drop(['UNIT_VARP','PRECISION_VALUE',
                      'PRECISION_VAR','PRECISION_VARP',
                      'LAST_UPDATE'], axis=1)
    else:
        print(f'DataFrame empty !')
    return df

if continue_script:
    df_france = transform_data(load_source())
    df_france.head(5)

DataFrame empty !
CPU times: user 982 µs, sys: 0 ns, total: 982 µs
Wall time: 5.51 ms


""


In [4]:
#Var France
map_center = {"lat": 46.7667, "lon": 2.45}
def create_map(df,geo,dfKey,geoKey,scale,colors,map_center,label,file_html):
    file_path = os.path.join(OUTPUT_FOLDER,file_html)
    fig = px.choropleth_mapbox(df, 
                               geojson=geo, 
                               color="VALUE",
                               locations=dfKey, 
                               featureidkey=geoKey,
                               range_color=scale,
                               color_continuous_scale=colors,
                               mapbox_style="carto-positron", 
                               zoom=5, 
                               center=map_center,
                               hover_data=["VALUE","VAR"],
                               labels=label,
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True)
    fig.write_html(file_path)
    updateChartCss(file_path, os.path.join(INPUT_FOLDER,'map.css'))
#     return fig

In [5]:
# MAP_WSR = pd.read_excel(REF_WSR_PATH, sheet_name="MAP_FRANCE")

# def generate_map(df):
#     for _, row in MAP_WSR.iterrows():
#         kpi = row['KPI']
#         print(kpi)
#         file_html = row['FILE_HTML']
#         print(file_html)
#         scale_min = row['SCALE_MIN']
#         scale_max = row['SCALE_MAX']
#         scale = (scale_min, scale_max)
#         print(scale)
#         colors_s1 = row['COLORS_S1']
#         colors_c1 = row['COLORS_C1']
#         colors_s2 = row['COLORS_S2']
#         colors_c2 = row['COLORS_C2']
#         colors_s3 = row['COLORS_S3']
#         colors_c3 = row['COLORS_C3']
#         colors_s4 = row['COLORS_S4']
#         colors_c4 = row['COLORS_C4']
#         colors_s5 = row['COLORS_S5']
#         colors_c5 = row['COLORS_C5']
#         colors = [[colors_s1,colors_c1],[colors_s2,colors_c2],[colors_s3,colors_c3],[colors_s4,colors_c4],[colors_s5,colors_c5]]
#         print(colors)
#         dfKey = row['DF_KEY']
#         print(dfKey)
#         geoKey = row['GEO_KEY']
#         print(geoKey)
#         to_rename = row['TO_RENAME']
#         print(to_rename)
        
#         df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
#         analysis(df_map,'VALUE')
        
#         create_map(df_map,MAP_FRANCE_DEP,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
#         print(token_hosp_dep)
        
# generate_map(df_france)

In [6]:
#Hospitalisation - Département
def generate_map_asset(kpi, filename, map_base, scale, colors):

    df_map = df_france.copy()[(df_france['UPPER_FILTER_M'] == kpi) & (df_france['PARENT'] != 'root')]
    analysis(df_map,'VALUE')

    dfKey = "LABEL"
    geoKey="properties.nom"
    to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
    create_map(df_map,map_base,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)

    #Make html visible
    return naas.assets.add(os.path.join(OUTPUT_FOLDER, file_html), params={"inline":True})


KeyError: 'UPPER_FILTER_M'

In [ ]:
#Hospitalisation - Département
scale = (0,3000)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
if continue_script:
    url_hosp_dep = generate_map_asset('Hospitalisations', "France_HOSP_DEP_NAME.html", MAP_FRANCE_DEP, scale, colors)

In [7]:
#En réanimation - Département
scale = (0,800)
colors = [
    [0.0, "#FFFFFF"],
    [0.025,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]

if continue_script:
    url_rea_dep = generate_map_asset('En réanimation', "France_REA_DEP_NAME.html", MAP_FRANCE_DEP, scale, colors)

KeyError: 'UPPER_FILTER_M'

In [ ]:
#"Décès à l’hôpital" - Département
scale = (0,1500)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]

if continue_script:
    url_dc_dep = generate_map_asset('Décès à l’hôpital', "France_DC_DEP_NAME.html", MAP_FRANCE_DEP, scale, colors)

In [ ]:
#'Retours à domicile' - Département
scale = (0,3000)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]

if continue_script:
    url_rad_dep = generate_map_asset('Retours à domicile', "France_RAD_DEP_NAME.html", MAP_FRANCE_DEP, scale, colors)

#ANALYSIS BY REGION

In [ ]:
#Hospitalisation - Région
scale = (100,13000)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]

if continue_script:
    url_hosp_reg = generate_map_asset('Hospitalisations', "France_HOSP_REG_NAME.html", MAP_FRANCE_REG, scale, colors)

In [ ]:
#En réanimation - Région
scale = (50,2500)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]

if continue_script:
    url_rea_reg = generate_map_asset('En réanimation', "France_REA_REG_NAME.html", MAP_FRANCE_REG, scale, colors)

In [ ]:
#"Décès à l’hôpital" - Région
scale = (100,5500)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]

if continue_script:
    url_dc_reg = generate_map_asset('Décès à l’hôpital', "France_DC_REG_NAME.html", MAP_FRANCE_REG, scale, colors)

In [ ]:
#Retours à domicile - Région
scale = (200,15000)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]

if continue_script:
    url_rad_reg = generate_map_asset('Retours à domicile', "France_RAD_REG_NAME.html", MAP_FRANCE_REG, scale, colors)

In [ ]:
map_center = {"lat": 46.7667, "lon": 2.45}
def create_map_epid(df,geo,dfKey,geoKey,col_color,hoverdata,map_center,label,file_html):
    fig = px.choropleth_mapbox(df, 
                               geojson=geo, 
                               color=col_color,
                               locations=dfKey, 
                               featureidkey=geoKey,
                                color_discrete_map={ # replaces default color mapping by value
                                    "vert": "#4CCD88", "orange": "#FDB47A", "rouge": "#DD6D88"
                                },
                               mapbox_style="carto-positron", 
                               zoom=5, 
                               center=map_center,
                               hover_data= hoverdata,
                               labels=label,
                               opacity=0.8
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True, showlegend=False)
    fig.write_html(os.path.join(OUTPUT_FOLDER,file_html))
    updateChartCss(os.path.join(OUTPUT_FOLDER,file_html), os.path.join(INPUT_FOLDER,'map.css'))
#     return fig

In [ ]:
#--- Situation epidémiologique DATA
if continue_script:
    cols = ['DEP','DEP_NAME','REA_2013','REA_2018','SI_2013','SI_2018','SC_2013','SC_2018','ALL_2013','ALL_2018']
    ref_france_lits = pd.read_excel(os.path.join(INPUT_FOLDER,'drees_lits_reanimation_2013-2018.xlsx'), sheet_name = 'Lits_2013-2018', header=7, names=cols)
    ref_france_lits = ref_france_lits[['DEP','DEP_NAME','REA_2018']]
    ref_france_lits['DEP'] = ref_france_lits['DEP'].astype(str)
    ref_france_lits.head(5)

In [ ]:
#---Tension Data
if continue_script:
    df_rea = pd.read_csv(os.path.join(OUTPUT_FOLDER,'107_FRANCE.csv'),sep=";").drop('Unnamed: 0',axis=1)
    df_rea = df_rea[['ENTITY','SCENARIO','CHILD','PARENT','UPPER_FILTER_M','VALUE','VAR']]
    df_rea = pd.merge(df_rea, REF_FRANCE, left_on='CHILD', right_on='DEP_FULL',how='left').rename(index=str, columns={"DEP_NAME":"LABEL"}).drop(['DEP_FULL'], axis=1)
    df_rea.loc[df_rea['PARENT'] == 'root', 'LABEL'] = df_rea['CHILD']
    df_rea['DEP_CODE'] = df_rea['DEP_CODE'].astype(str)

    df_rea.loc[df_rea['CHILD'] == 'France', 'DEP_CODE'] = 'FR'
    df_rea = pd.merge(df_rea,ref_france_lits, left_on='DEP_CODE', right_on='DEP',how='left').drop(['DEP','DEP_NAME'], axis=1)
    df_rea.head(5)

In [ ]:
#---HKPIS
if continue_script:
    df_hkpis = df_rea.copy()[(df_rea['CHILD'] == 'France') & (df_rea['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y'))]
    df_kpis = df_hkpis[['ENTITY','SCENARIO','UPPER_FILTER_M','VALUE','VAR']]

    df_reakpi = df_hkpis.copy()[df_hkpis['UPPER_FILTER_M'] == 'En réanimation'][['ENTITY','SCENARIO','UPPER_FILTER_M','VALUE','VAR','REA_2018']]
    df_reakpi['VALUEP'] = round((df_reakpi['VALUE'] / df_reakpi['REA_2018'] * 100),2)
    df_reakpi['VALUE_N1'] = (df_reakpi['VALUE'] - df_reakpi['VAR'])
    df_reakpi['VALUEP_N1'] = round(((df_reakpi['VALUE_N1'] / df_reakpi['REA_2018']) * 100),2)
    df_reakpi['VALUEP_VAR'] = df_reakpi['VALUEP'] - df_reakpi['VALUEP_N1']

    df_reakpi = df_reakpi.drop(['VALUE','VAR','REA_2018','VALUE_N1','VALUEP_N1'],axis=1).rename(index=str, columns={"VALUEP":"VALUE","VALUEP_VAR":"VAR"})
    df_reakpi['UPPER_FILTER_M'] = 'Tensions hospitalières'

    df_final = pd.concat([df_kpis,df_reakpi], axis=0)
    df_final.loc[df_final['UPPER_FILTER_M'] == 'Tensions hospitalières','UNIT_VALUE'] = ' %'
    df_final.loc[df_final['UPPER_FILTER_M'] == 'Tensions hospitalières','UNIT_VAR'] = ' pts'
    df_final.loc[:,'PRECISION'] = ',.0f'
    df_final.loc[df_final['UPPER_FILTER_M'] == 'Tensions hospitalières','PRECISION'] = ',.2f'

    if USE_MONGO:
        naas_drivers.mongo.send(df_final,'110_BIS',DB_APP,True)
    df_final

In [ ]:
#Par département
if continue_script:
    df_rea_dep = df_rea.copy()[(df_rea['UPPER_FILTER_M'] == 'En réanimation') & (df_rea['PARENT'] != 'root') & (df_rea['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y'))]
    df_rea_dep['VALUEP'] = round((df_rea_dep['VALUE'] / df_rea_dep['REA_2018'] * 100),2)
    df_rea_dep.loc[df_rea_dep['VALUEP'] < 60 ,'COLOR'] = 'vert'
    df_rea_dep.loc[df_rea_dep['VALUEP'] >= 60, 'COLOR'] = 'orange'
    df_rea_dep.loc[df_rea_dep['VALUEP'] >= 80, 'COLOR'] = 'rouge'
    df_rea_dep.head(5)

In [ ]:
#---Tension_Département
if continue_script:
    kpi = 'Tension hospitalière (%)'
    file_html = "France_TSHOSP_DEP_NAME.html"
    dfKey = "LABEL"
    geoKey="properties.nom"
    col_color = 'COLOR'
    hoverdata=["VALUE","VAR","REA_2018",'VALUEP']
    to_rename = {'COLOR': "Niveau",'LABEL':'Département', 'VALUE': 'Nombre de cas en réanimation', 'VAR': 'Evolution en réanimation','REA_2018': 'Nombre de lits en service réanimation','VALUEP': kpi}
    create_map_epid(df_rea_dep,MAP_FRANCE_DEP,dfKey,geoKey,col_color,hoverdata,map_center,to_rename,file_html)

    #Make html visible
    url_th_dep = naas.assets.add(os.path.join(OUTPUT_FOLDER, file_html), params={"inline":True})

In [ ]:
#Par région
if continue_script:
    df_rea_re = df_rea.copy()[(df_rea['UPPER_FILTER_M'] == 'En réanimation') & (df_rea['PARENT'] != 'root') & (df_rea['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y'))]
    cols_to_group = ['ENTITY','SCENARIO','REG_NAME']
    df_rea_re['VALUE'] = df_rea_re['VALUE'].astype(int)
    df_rea_re['VAR'] = df_rea_re['VAR'].astype(int)
    df_rea_re = df_rea_re.groupby(cols_to_group, as_index=False).agg({"VALUE":'sum',"VAR":'sum',"REA_2018":'sum'})

    df_rea_re['VALUEP'] = round((df_rea_re['VALUE'] / df_rea_re['REA_2018'] * 100),2)
    df_rea_re.loc[df_rea_re['VALUEP'] < 60 ,'COLOR'] = 'vert'
    df_rea_re.loc[df_rea_re['VALUEP'] >= 60, 'COLOR'] = 'orange'
    df_rea_re.loc[df_rea_re['VALUEP'] >= 80, 'COLOR'] = 'rouge'
    df_rea_re.head(5)

In [ ]:
#---Tension_Région
if continue_script:
    kpi = 'Tension hospitalière (%)'
    file_html = "France_TSHOSP_REG_NAME.html"
    dfKey = "REG_NAME"
    geoKey="properties.nom"
    col_color = 'COLOR'
    hoverdata=["VALUE","VAR","REA_2018",'VALUEP']
    to_rename = {'REG_NAME':'Région','COLOR': "Niveau", 'VALUE': 'Nombre de cas en réanimation','VAR': 'Evolution en réanimation', 'REA_2018': 'Nombre de lits en service réanimation','VALUEP': kpi}
    create_map_epid(df_rea_re,MAP_FRANCE_REG,dfKey,geoKey,col_color,hoverdata,map_center,to_rename,file_html)

    #Make html visible
    url_th_reg = naas.assets.add(os.path.join(OUTPUT_FOLDER, file_html), params={"inline":True})

In [ ]:
# html
# Par département
if continue_script:

    HOSP_DEP = [{'URL': url_hosp_dep,
                 'KPI': 'Hospitalisations',
                 'UPPER_FILTER': 'Par département'}]

    REA_DEP = [{'URL': url_rea_dep,
                'KPI': 'En réanimation',
                'UPPER_FILTER': 'Par département'}]

    DC_DEP = [{'URL': url_dc_dep,
               'KPI': 'Retours à domicile',
               'UPPER_FILTER': 'Par département'}]

    RAD_DEP = [{'URL': url_rad_dep,
                'KPI': 'Décès à l’hôpital',
                'UPPER_FILTER': 'Par département'}]

    REAT_DEP = [{'URL': url_th_dep,
                 'KPI': 'Tensions hospitalières',
                 'UPPER_FILTER': 'Par département'}]

    # Par région
    HOSP_REG = [{'URL': url_hosp_reg,
                 'KPI': 'Hospitalisations',
                 'UPPER_FILTER': 'Par région'}]

    REA_REG = [{'URL': url_rea_reg,
                'KPI': 'En réanimation',
                'UPPER_FILTER': 'Par région'}]

    DC_REG = [{'URL': url_dc_reg,
               'KPI': 'Retours à domicile',
               'UPPER_FILTER': 'Par région'}]

    RAD_REG = [{'URL': url_rad_reg,
                'KPI': 'Décès à l’hôpital',
                'UPPER_FILTER': 'Par région'}]

    REAT_REG = [{'URL': url_th_reg,
                 'KPI': 'Tensions hospitalières',
                 'UPPER_FILTER': 'Par région'}]

    # Concat list to create Series
    LIST_KPIS = (HOSP_DEP + REA_DEP + DC_DEP + RAD_DEP +
                 HOSP_REG + REA_REG + DC_REG + RAD_REG + 
                 REAT_DEP + REAT_REG)

    # Create Dataframe
    DOMAIN = pd.DataFrame(LIST_KPIS)
    DOMAIN.loc[:, 'VALUE'] = ("<iframe src='" +
                              DOMAIN['URL'] +
                              ("' frameborder='0' style='overflow:hidden;"
                               "height:85vh;width:100%' height='100%' "
                               "width='100%'></iframe> <style>.modebar"
                               "{display: none!important;}</style>"))
    if USE_MONGO:
        naas_drivers.mongo.send(DOMAIN,'110',DB_APP,True)
    DOMAIN